# task collection aggrigation and check

In [1]:
import multiprocessing
from connections import get_mongo_client
from my_utils import get_numbers

In [2]:
client = get_mongo_client()
db = client['task']
collection = db['task']

In [31]:
indexes = collection.index_information()
indexes

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'expire_1': {'v': 2, 'key': [('expire', 1)], 'background': True},
 'modified_1': {'v': 2, 'key': [('modified', 1)], 'background': True},
 'asbt.status_1': {'v': 2, 'key': [('asbt.status', 1)], 'background': True},
 'type_1': {'v': 2, 'key': [('type', 1)], 'background': True},
 'created_1': {'v': 2, 'key': [('created', 1)], 'background': True},
 'result.error.errmsg_1': {'v': 2,
  'key': [('result.error.errmsg', 1)],
  'background': True},
 'result.is_ok_1': {'v': 2, 'key': [('result.is_ok', 1)], 'background': True},
 'number_1': {'v': 2, 'key': [('number', 1)], 'background': True},
 'status_1': {'v': 2, 'key': [('status', 1)], 'background': True},
 'status_1_created_1': {'v': 2,
  'key': [('status', 1), ('created', 1)],
  'background': True},
 'result.is_ok_1_created_1': {'v': 2,
  'key': [('result.is_ok', 1), ('created', 1)],
  'background': True},
 'asbt.status_1_created_1': {'v': 2,
  'key': [('asbt.status', 1), ('created', 1)],
  'background

In [34]:
collection.find({'data.katm_077': {'$exists': True}}).explain()

{'queryPlanner': {'plannerVersion': 1,
  'namespace': 'task.task',
  'indexFilterSet': False,
  'parsedQuery': {'data.katm_077': {'$exists': True}},
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'data.katm_077': {'$exists': True}},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 3658804,
  'executionTimeMillis': 1219103,
  'totalKeysExamined': 0,
  'totalDocsExamined': 7637143,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'data.katm_077': {'$exists': True}},
   'nReturned': 3658804,
   'executionTimeMillisEstimate': 1205537,
   'works': 7637145,
   'advanced': 3658804,
   'needTime': 3978340,
   'needYield': 0,
   'saveState': 76797,
   'restoreState': 76797,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 7637143},
  'allPlansExecution': []},
 'serverInfo': {'host': 'crm-clone-risk.ipa.ipotekabank.uz',
  'port': 27017,
  'version': '4.4.29',
  'gitVersion': 'f4dda329a99811c707eb06d05ad0235

### count total number of documents aka applications

In [4]:
total_sysdocs = collection.count_documents({'data.katm_077.return.data.sysinfo': {'$exists': True}})
print(f"Number of sysdocs: {total_sysdocs}")

Number of sysdocs: 3583379


In [5]:
total_docs = collection.count_documents({'data.katm_077': {'$exists': True}})
print(f"Number of documents: {total_docs}")

Number of documents: 3563906


### count number of all overview fields

In [13]:
total_overview_fields = collection.count_documents({'data.katm_077.return.da ta.overview': {'$exists': True}})
print(f"total overviews: {total_overview_fields}")

total overviews: 3424620


##### check if index is being used in above query

In [27]:
collection.find({'data.katm_077.return.data.overview': {'$exists': True}}).explain(verbosity='queryPlanner')

TypeError: Cursor.explain() got an unexpected keyword argument 'verbosity'

##### use indexing

In [17]:
collection.create_index([('data.katm_077.return.data.overview', 1)])

OperationFailure: not authorized on task to execute command { createIndexes: "task", indexes: [ { name: "data.katm_077.return.data.overview_1", key: { data.katm_077.return.data.overview: 1 } } ], lsid: { id: UUID("ddf93f55-0a21-409e-9fa3-cf121996b952") }, $db: "task" }, full error: {'ok': 0.0, 'errmsg': 'not authorized on task to execute command { createIndexes: "task", indexes: [ { name: "data.katm_077.return.data.overview_1", key: { data.katm_077.return.data.overview: 1 } } ], lsid: { id: UUID("ddf93f55-0a21-409e-9fa3-cf121996b952") }, $db: "task" }', 'code': 13, 'codeName': 'Unauthorized'}

#### get numbers in mongo overview fields

In [11]:
docs = collection.find({'data.katm_077.return.data.overview': {'$exists': True}})

In [12]:
mongo_total_numbers = [doc['number'] for doc in docs if 'number' in doc]

KeyboardInterrupt: 

In [18]:
def process_documents(doc_queue, result_queue):
    while True:
        doc = doc_queue.get()
        if doc is None: 
            break
        if 'number' in doc:
            result_queue.put(doc['number'])

In [21]:
doc_queue = multiprocessing.Queue(maxsize=10)
result_queue = multiprocessing.Queue()

# Start worker processes
num_workers = multiprocessing.cpu_count()
workers = []
for _ in range(num_workers):
    p = multiprocessing.Process(target=process_documents, args=(doc_queue, result_queue))
    p.start()
    workers.append(p)

# Feed documents into the queue
for doc in docs:
    doc_queue.put(doc)

# Send sentinel values to signal the end
for _ in range(num_workers):
    doc_queue.put(None)

# Collect results
mongo_total_numbers = []
while any(p.is_alive() for p in workers) or not result_queue.empty():
    while not result_queue.empty():
        mongo_total_numbers.append(result_queue.get())

# Wait for all worker processes to finish
for p in workers:
    p.join()

print(f"there are {len(mongo_total_numbers)} in mongo_total_numbers list")

KeyboardInterrupt: 

In [ ]:
print(f"there are {len(mongo_total_numbers)} overview fields in mongodb")

#### juxtapose

In [ ]:
sql_total_overview_numbers = get_numbers('bronze.katm_077_overview')

In [ ]:
print(f"there are {len(sql_total_overview_numbers)} numbers in sql server")

#### missing numbers from sql

In [ ]:
sql_total_overview_numbers = [int(i) for i in sql_total_overview_numbers]

In [ ]:
missing_numbers = list(set(mongo_total_numbers)-set(sql_total_overview_numbers))

In [ ]:
print(f"there are len({missing_numbers}) numbers missing in sql overview table")

### count total number of contracts

In [22]:
total_contracts_fields = collection.count_documents({'data.katm_077.return.data.contracts.contract': {'$exists': True}})
print(f"total_contracts_fields: {total_contracts_fields}")

total contracts: 3425892


### count total number of open contracts

In [23]:
total_open_contracts_fields = collection.count_documents({'data.katm_077.return.data.open_contracts.open_contract': {'$exists': True}})
print(f"total_open_contracts_fields: {total_open_contracts_fields}")

total contracts: 3153516


In [24]:
total_contingent_liabilities_fields = collection.count_documents({'data.katm_077.return.data.contingent_liabilities.contingent_liability': {'$exists': True}})
print(f"total_contingent_liabilities_fields: {total_contingent_liabilities_fields}")

total contracts: 3427417


In [25]:
total_scores = collection.count_documents({'data.katm_077.return.data.scorring': {'$exists': True}})
print(f"total_scores: {total_scores}")

total_scores: 3428346


In [26]:
total_scores = collection.count_documents({'data.katm_077.return.data.scorring.scoring_grade': {'$exists': True}})
print(f"total_scores: {total_scores}")

total_scores: 3428346
